# Term Project
## Synthesizing Raw Audio Signals to Sound like Saxophone Sounds

In [ ]:
# Utility Functions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

The function get_music_notes() maps the frequency in hertz to each key on a standard saxophone. Within the function, a list named octave holds the names of keys in musical notation, distinguishing between uppercase for natural notes and lowercase for their corresponding sharps or flats. Setting the base_freq variable to 440 Hz establishes a reference point, specifically the frequency of the note A4. The function then constructs an array called keys, encompassing all possible combinations of note names and octave numbers across the saxophone's range. It then prunes this array to adhere to the standard 88 key piano scale, ensuring compatibility with conventional musical instruments. The function then generates note_freqs, a dictionary mapping each key to its respective frequency, calculated according to the equal tempered scale formula, which maintains consistent intervals between adjacent notes. Additionally, a placeholder with a frequency of 0.0 is appended to the dictionary, serving as a termination signal.

In [ ]:
def get_music_notes():
    '''
    Get the frequency in hertz for all keys on a standard saxophone.

    Returns
    -------
    note_freqs : dict
        Mapping between note name and corresponding frequency.

    '''

    # Major keys in capital letter and
    octave = ['C', 'c', 'D', 'd', 'E', 'F', 'f', 'G', 'g', 'A', 'a', 'B']
    base_freq = 440 #Frequency of Note A4
    keys = np.array([x+str(y) for y in range(0,9) for x in octave])
    # Trim to standard 88 scale
    start = np.where(keys == 'A0')[0][0]
    end = np.where(keys == 'C8')[0][0]
    keys = keys[start:end+1]

    note_freqs = dict(zip(keys, [2**((n+1-49)/12)*base_freq for n in range(len(keys))]))
    note_freqs[''] = 0.0 # stop
    return note_freqs

The function get_sine_wave() generates pure sine. The function takes several parameters, including the desired frequency of the sine wave (frequency), its duration (duration), and optional parameters such as the sample rate (sample_rate) and peak amplitude (amplitude). Inside the function, a time array t is created using np.linspace() to span the duration of the waveform, with the number of samples determined by the product of the sample rate and duration. The sine wave itself is then generated using np.sin(), with its amplitude scaled by the specified amplitude parameter. This results in a waveform that oscillates at the given frequency over the specified duration.


In [ ]:
def get_sine_wave(frequency, duration, sample_rate=44100, amplitude=4096):
    '''
    Get pure sine wave.

    Parameters
    ----------
    frequency : float
        Frequency in hertz.
    duration : float
        Time in seconds.
    sample_rate : int, optional
        Wav file sample rate. The default is 44100.
    amplitude : int, optional
        Peak Amplitude. The default is 4096.

    Returns
    -------
    wave : TYPE
        DESCRIPTION.

    '''
    t = np.linspace(0, duration, int(sample_rate*duration))
    wave = amplitude*np.sin(2*np.pi*frequency*t)
    return wave

The function apply_saxophone_overtones() mimics the timbre of a saxophone by blending overtones with a fundamental frequency. Its parameters include the fundamental frequency (frequency), the duration of the resulting sound (duration), a list of relative amplitudes for overtones (sax_factor), and optional settings such as sample rate (sample_rate) and amplitude (amplitude). Inside the function, it computes frequencies for overtones based on the provided saxophone factors, ensuring they don't exceed half the sample rate to prevent distortion. Corresponding amplitudes for overtones are also determined, scaled by the saxophone factors. The fundamental sine wave is first generated, then subsequent overtones are added iteratively to it. This process forms a composite waveform representing the saxophone like sound.

In [ ]:
def apply_saxophone_overtones(frequency, duration, sax_factor, sample_rate=44100, amplitude=4096):

    frequencies = np.minimum(np.array([frequency * (x - 5) for x in range(len(sax_factor))]), sample_rate // 2)
    amplitudes = np.array([amplitude * (x * 2.5) for x in sax_factor])

    fundamental = get_sine_wave(frequencies[0], duration, sample_rate, amplitudes[0])
    for i in range(1, len(sax_factor)):
        overtone = get_sine_wave(frequencies[i], duration, sample_rate, amplitudes[i])
        fundamental += overtone

    return fundamental

The apply_adsr_envelope() function shapes the amplitude of the audio waveform over time using an ADSR (Attack, Decay, Sustain, Release) envelope. With parameters including the input waveform, sample rate, and durations for each phase of the envelope, it provides fine grained control over the dynamic characteristics of the audio signal. Inside the function, the lengths of each envelope phase are calculated in samples based on the provided times and sample rate. Subsequently, an empty array is initialised to store the ADSR envelope. The envelope is then constructed by smoothly transitioning between different amplitude levels during the attack, decay, and release phases, while maintaining a constant level during the sustain phase. Finally, the envelope is applied to the input waveform through element wise multiplication, resulting in a modulated waveform that exhibits the desired amplitude changes over time.

In [ ]:
import numpy as np

def apply_adsr_envelope(input_wave, sample_rate, attack_time, decay_time, sustain_level, release_time):
    """
    Apply an ADSR envelope to the input wave.

    Args:
    - input_wave (numpy array): The wave to modulate.
    - sample_rate (int): The sample rate of the audio signal.
    - attack_time (float): Time for the attack phase in seconds.
    - decay_time (float): Time for the decay phase in seconds.
    - sustain_level (float): Amplitude level during the sustain phase, relative to peak.
    - release_time (float): Time for the release phase in seconds.

    Returns:
    - numpy array: The wave modulated by the ADSR envelope.
    """
    num_samples = len(input_wave)
    attack_samples = int(sample_rate * attack_time)
    decay_samples = int(sample_rate * decay_time)
    release_samples = int(sample_rate * release_time)

    # ADSR Envelope Construction
    envelope = np.zeros(num_samples)

    # Attack
    if attack_samples > 0:
        envelope[:attack_samples] = np.linspace(0, 1, attack_samples)

    # Decay
    decay_start = attack_samples
    decay_end = decay_start + decay_samples
    if decay_samples > 0:
        envelope[decay_start:decay_end] = np.linspace(1, sustain_level, decay_samples)

    # Sustain
    sustain_start = decay_end
    sustain_end = num_samples - release_samples
    envelope[sustain_start:sustain_end] = sustain_level

    # Release
    if release_samples > 0:
        envelope[sustain_end:] = np.linspace(sustain_level, 0, release_samples)

    # Apply envelope to the input wave
    modulated_wave = input_wave * envelope

    return modulated_wave



The compute_envelope_lowpass() function derives the amplitude envelope of the input audio signal through low pass filtering. It accepts parameters including the input signal, sample rate, cutoff frequency, and filter order. Using SciPy's butter() function, it computes the coefficients for a low pass Butterworth filter based on the provided cutoff frequency and filter order. The absolute value of the input signal is then passed through the filter using filtfilt(), resulting in a smoothed amplitude envelope that captures the overall variations in signal amplitude while attenuating high frequency components beyond the cutoff frequency.

In [ ]:
from scipy.signal import butter, filtfilt

def compute_envelope_lowpass(signal, sample_rate, cutoff_freq=10, order=4):
    """
    Compute the amplitude envelope of a signal using low-pass filtering.

    Args:
    - signal (numpy array): The input audio signal.
    - sample_rate (int): The sample rate of the audio signal.
    - cutoff_freq (float): Cutoff frequency for the low-pass filter.
    - order (int): Order of the filter.

    Returns:
    - numpy array: The smoothed amplitude envelope of the signal.
    """
    b, a = butter(order, cutoff_freq / (0.5 * sample_rate), btype='low')
    envelope = filtfilt(b, a, np.abs(signal))  # Apply the filter to the rectified signal
    return envelope


The apply_amplitude_envelope() function facilitates the application of an amplitude envelope to the input audio waveform. Upon receiving the input waveform and the desired amplitude envelope, the function first ensures their lengths match to maintain consistency in the modulation process. Subsequently, it applies the envelope to the waveform through element wise multiplication, effectively scaling each sample of the waveform according to the corresponding values in the envelope array. This operation results in a modulated waveform where the amplitude evolves over time as dictated by the envelope.

In [ ]:
def apply_amplitude_envelope(input_wave, envelope):
    """
    Apply an amplitude envelope to the input wave.

    Args:
    - input_wave (numpy array): The wave to modulate.
    - envelope (numpy array): The amplitude envelope array.

    Returns:
    - numpy array: The amplitude-modulated wave.
    """
    # Ensure the envelope is the same length as the input wave
    if len(input_wave) != len(envelope):
        raise ValueError("The input wave and envelope must be the same length.")

    # Apply the envelope to the wave
    modulated_wave = input_wave * envelope

    return modulated_wave
